In [ ]:
from Account import Account
from Clock import Clock
from Property import Property
from StockInvestment import StockInvestment
from plotting import PlotSeries, plot
from config import (
    INITIAL_CASH,
    SALARY,
    SALARY_INCREASE,
    PROPERTY_AREA,
    TIME_FRAME_IN_YEARS,
)


valueSeriesList = []
stockValueSeriesList = []
propValueSeriesList = []
for house_purchase_delay in range(0, 60, 10):
    legendText = (
        "House purchase after " + str(house_purchase_delay) + " Years"
        if house_purchase_delay > 0
        else "House purchase immediately"
    )
    valueSeries = PlotSeries(legendText)
    stockValueSeries = PlotSeries(legendText)
    propValueSeries = PlotSeries(legendText)
    clock = Clock()
    stock_inv = StockInvestment(clock)
    prop = Property(clock, PROPERTY_AREA)
    account = Account(INITIAL_CASH)
    for i in range(TIME_FRAME_IN_YEARS * 12 + 2):
        account.deposit(SALARY * (1 + SALARY_INCREASE) ** (i / 12))
        prop.pay_costs(account)
        if i == house_purchase_delay * 12:
            required_additional_cash = prop.get_total_value() - account.get_balance()
            stock_inv.attempt_withdrawal(account, required_additional_cash)
            prop.buy(account)
        stock_inv.deposit(account, -1)

        valueSeries.put_value(
            (stock_inv.get_value() + prop.get_owned_value()) / 1000000
        )
        stockValueSeries.put_value(stock_inv.get_value() / 1000000)
        propValueSeries.put_value(prop.get_owned_value() / 1000000)
        clock.tick()
    valueSeriesList.append(valueSeries)
    stockValueSeriesList.append(stockValueSeries)
    propValueSeriesList.append(propValueSeries)

plot(
    {
        "title": "Total Value over Time",
        "xlabel": "Year",
        "ylabel": "Value in Mio €",
        "series": valueSeriesList,
    }
)
plot(
    {
        "title": "Stock Value over Time",
        "xlabel": "Year",
        "ylabel": "Value in Mio €",
        "series": stockValueSeriesList,
    }
)
plot(
    {
        "title": "Property Value over Time",
        "xlabel": "Year",
        "ylabel": "Value in Mio €",
        "series": propValueSeriesList,
    }
)